# Linear Regression - un projet de consulting

félicitations! Hyundai Heavy Industries vous a confié la tâche de les aider à élaborer un modèle prédictif pour certains navires. [Hyundai Heavy Industries](http://www.hyundai.eu/en) est l’une des plus grandes entreprises de fabrication de navires au monde et construit des navires de croisière.

Vous avez été transféré par avion à leur quartier général situé à Ulsan, en Corée du Sud, pour les aider à évaluer avec précision le nombre de membres d'équipage nécessaires à un navire.

Ils construisent actuellement de nouveaux navires pour certains clients et souhaitent que vous créiez un modèle et l’utilisiez pour prédire le nombre de membres d’équipage "crew members" dont les navires auront besoin.

Voici à quoi ressemblent les données jusqu'à présent:

    Description: Mesures de la taille, de la capacité, de l'équipage et de l'âge du navire pour une croisière de 158
    navires.


    Variables
    Ship Name     
    Cruise Line  
    Age (as of 2013)   
    Tonnage (1000s of tons)   
    passengers (100s)   
    Length (100s of feet)  
    Cabins  (100s)   
    Passenger Density   
    Crew  (100s)   
    
Il est enregistré dans un fichier csv appelé "cruise_ship_info.csv". Votre travail consiste à créer un modèle de régression permettant de prévoir le nombre de membres d'équipage nécessaires pour les futurs navires. Le client a également indiqué qu'il avait constaté que le nombre d'équipages accepté par les lignes de croisières différait d'un critère de sélection acceptable; il est donc très probablement une caractéristique importante à inclure dans votre analyse!

Une fois que vous avez créé le modèle et l'avoir testé pour vérifier rapidement son efficacité, assurez-vous de bien comprendre pourquoi il fonctionne si bien!

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_consulting').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [6]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [7]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [8]:
data.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)

In [9]:
for item in data.head():
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [10]:
# comme dans le notebook precendent l'API Spark ML doit recevoir en entrée un dataframe !
# de cette forme 
# ("label","features")
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [13]:
assembler = VectorAssembler(
    inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density'],
    outputCol="features")

In [14]:
output = assembler.transform(data)

In [15]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [16]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|[17.0,101.353,26....|
|    Ecstasy|   Carnival| 22|            70.367|     20.

In [17]:
final_data = output.select("features",'crew')

In [18]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [19]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               113|
|   mean|7.8258407079646135|
| stddev|3.5942103060894324|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [20]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                45|
|   mean| 7.714666666666667|
| stddev|3.3023095224127887|
|    min|               0.6|
|    max|              13.6|
+-------+------------------+



In [21]:
# créer un modèle de régression 
lr = LinearRegression(labelCol='crew')

In [22]:
lrModel = lr.fit(train_data,)

In [23]:
# afficher les coéfficients et l'interception 
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.0149358876391162,0.01184675404852862,-0.1452270878487939,0.33817542387551747,0.8296252374922012,0.0011069438780692484] Intercept: -0.26094486196482153


In [24]:
test_results = lrModel.evaluate(test_data)

In [25]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.4513796927359133|
|  0.3324366979562612|
|-0.35255809696588614|
| -0.6814227396051695|
|  0.9445986166716338|
|  -0.105965489629674|
| -0.5205720614712703|
| -0.2535195391380469|
|-0.39077352677644406|
|  1.6515885989011103|
|-0.32102738798141317|
|-0.25266279480851905|
|-0.09917867106095546|
|   0.792376286509775|
|  0.7027629350097921|
| 0.11097427368870605|
|  -1.238019053411172|
|  0.2567371630534243|
|0.029800260023339575|
|  -1.224381699244371|
+--------------------+
only showing top 20 rows



In [26]:
unlabeled_data = test_data.select('features')

In [27]:
predictions = lrModel.transform(unlabeled_data)

In [28]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[6.0,30.276999999...| 4.001379692735913|
|[6.0,110.23899999...|11.167563302043739|
|[6.0,158.0,43.7,1...|13.952558096965886|
|[7.0,116.0,31.0,9...| 12.68142273960517|
|[8.0,91.0,22.44,9...|10.055401383328366|
|[9.0,59.058,17.0,...| 7.505965489629674|
|[9.0,105.0,27.2,8...| 11.20057206147127|
|[9.0,116.0,26.0,9...|11.253519539138047|
|[10.0,86.0,21.14,...| 9.590773526776443|
|[10.0,151.4,26.2,...|10.878411401098889|
|[11.0,86.0,21.24,...| 9.621027387981414|
|[12.0,2.329,0.94,...| 0.852662794808519|
|[12.0,50.0,7.0,7....| 4.549178671060956|
|[12.0,77.104,20.0...| 8.797623713490225|
|[12.0,88.5,21.24,...| 9.587237064990207|
|[12.0,108.865,27....|10.889025726311294|
|[12.0,138.0,31.14...|13.088019053411172|
|[13.0,101.509,27....|11.243262836946576|
|[14.0,33.0,4.9,5....|3.2101997399766606|
|[14.0,63.0,14.4,7...| 6.834381699244371|
+--------------------+------------

In [29]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 0.9214275827884661
MSE: 0.8490287903233956
